<a href="https://colab.research.google.com/github/ThomasMcDaniel91/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_231_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [1]:
from google.colab import files
uploaded = files.upload()

Saving video-game-sales-with-ratings.zip to video-game-sales-with-ratings.zip


In [0]:
!pip install category_encoders==2.*
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import category_encoders as ce


In [27]:
df = pd.read_csv('/content/video-game-sales-with-ratings.zip')
print(df.shape)
df.head()

(16719, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#trying a different route with oridinal encoding and removing all rows except 
#completely filled ones(6,825 rows remaining of 16,719)
df = df.dropna()
df.shape

(6825, 16)

In [39]:
#getting a baseline with the new nan policy(1.112)MAE
df['User_Score'] = pd.to_numeric(df['User_Score'])
user_mean = df['User_Score'].mean()
user_mean_guesses = [user_mean] * len(df['User_Score'])
baseline = mean_absolute_error(df['User_Score'], user_mean_guesses)
baseline

1.1123492525057288

In [0]:
#getting rid of the years that have less than 15 entries after removing nans
threshold = 15
value_counts = df['Year_of_Release'].value_counts()
under_thresh = value_counts[value_counts <= threshold].index
df['Year_of_Release'].replace(under_thresh, np.nan, inplace=True)

In [58]:
#checking the value counts of the remaining years and removing the newly created
#nans from the cleaned column and checking shape again(removed 24 rows)
df['Year_of_Release'].value_counts()
df = df.dropna()
print(df.shape)
df.head()

(6801, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [60]:
target = 'User_Score'
features = df.columns.drop([target, 'Name', 'Global_Sales'])
features

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count',
       'User_Count', 'Developer', 'Rating'],
      dtype='object')

In [62]:
df['Year_of_Release'].value_counts()

2008.0    592
2007.0    590
2005.0    562
2009.0    550
2006.0    528
2003.0    498
2004.0    476
2002.0    455
2011.0    453
2010.0    429
2012.0    313
2013.0    266
2001.0    256
2014.0    253
2016.0    212
2015.0    211
2000.0    102
1999.0     30
1998.0     25
Name: Year_of_Release, dtype: int64

In [0]:
test = df[df['Year_of_Release'] >= 2014]
train = df[df['Year_of_Release'] < 2014]
train, val = train_test_split(train, train_size=0.75, random_state=1)

In [67]:
#displaying shape of the new splits.
#4593 rows for train, 1532 for val and 676 for test

#assigning the features and target for each split

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]
train.shape, val.shape, test.shape

((4593, 16), (1532, 16), (676, 16))

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    LinearRegression()
)

In [85]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Platform', 'Genre', 'Publisher',
                                      'Developer', 'Rating'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Platform',
                                          'data_type': dtype('O'),
                                          'mapping': PS3      1
PSP      2
XB       3
GC       4
X360     5
XOne     6
GBA      7
PS2      8
DS       9
Wii     10
PC      11
3DS     12
PS      13
PSV     14
PS4     15
DC      16
WiiU    17
NaN     -2
dtype: int64},
                                         {'col': 'Genr...
Length: 1076, dtype: int64},
                                         {'col': 'Rating',
                                          'data_type': dtype('O'),
                                          'mapping': T       1
E    

In [88]:
#accuracy improved from 1.112 to a MAE of .794
#which is an improvement from the previous model of 0.897
y_pred = pipeline.predict(X_val)
val_acc = mean_absolute_error(y_val, y_pred)
val_acc

0.7939686132636627